# Text Splitters trong LangChain

## Tại sao cần chia văn bản?

Khi làm việc với LLMs và RAG systems, chúng ta thường gặp phải những thách thức sau:

### 1. **Giới hạn Context Window**
- Hầu hết LLMs có giới hạn về số tokens có thể xử lý trong một lần
- Ví dụ: GPT-3.5 (~4K tokens), GPT-4 (~8K-32K tokens), Claude-3 (~200K tokens)
- Documents dài có thể vượt quá giới hạn này

### 2. **Hiệu quả Retrieval**
- Khi search trong vector database, chunks nhỏ hơn thường cho kết quả chính xác hơn
- Chunks lớn có thể chứa nhiều topics, làm giảm precision

### 3. **Cost Optimization**
- Chỉ retrieve và process những phần relevant
- Giảm số tokens gửi lên LLM

### 4. **Semantic Coherence**
- Mỗi chunk nên chứa một ý tưởng hoặc concept hoàn chỉnh
- Tránh cắt ngang giữa câu hoặc paragraph

Text Splitters giúp giải quyết những vấn đề này bằng cách chia documents thành những chunks có kích thước phù hợp.

## Setup và chuẩn bị dữ liệu

In [ ]:
# Import các thư viện cần thiết
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    TokenTextSplitter,
    SpacyTextSplitter,
    NLTKTextSplitter
)
from langchain.schema import Document
import tiktoken
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Tạo sample text cho demonstration
sample_text = """Giới thiệu về RAG (Retrieval-Augmented Generation)

RAG là một kỹ thuật kết hợp khả năng truy xuất thông tin (retrieval) với khả năng sinh văn bản (generation) của các Large Language Models. Phương pháp này giúp cải thiện đáng kể chất lượng và độ chính xác của các câu trả lời được tạo ra.

Kiến trúc RAG System

Một hệ thống RAG điển hình bao gồm các thành phần chính sau:

1. Document Loading và Preprocessing
Bước đầu tiên là tải và xử lý các documents từ nhiều nguồn khác nhau như PDF, websites, databases. Quá trình này bao gồm việc làm sạch dữ liệu, loại bỏ các ký tự đặc biệt, và chuẩn hóa format.

2. Text Splitting
Documents được chia thành các chunks nhỏ hơn để dễ dàng xử lý và tìm kiếm. Việc chia này phải đảm bảo tính toàn vẹn về mặt ngữ nghĩa và tránh cắt ngang giữa các ý tưởng quan trọng.

3. Embedding Generation
Mỗi chunk text được chuyển đổi thành vector embeddings sử dụng các mô hình như OpenAI Embeddings, Sentence-BERT, hoặc các mô hình embedding khác. Embeddings này capture được semantic meaning của text.

4. Vector Storage
Các embeddings được lưu trữ trong vector databases như Pinecone, Chroma, FAISS, hoặc Weaviate. Các databases này được tối ưu hóa cho việc tìm kiếm similarity.

5. Query Processing
Khi user đưa ra một câu hỏi, query cũng được convert thành embedding vector sử dụng cùng model đã dùng để embed documents.

6. Similarity Search
Vector database thực hiện similarity search để tìm ra các chunks có embedding gần nhất với query embedding. Thường sử dụng cosine similarity hoặc euclidean distance.

7. Context Construction
Các chunks relevant nhất được kết hợp lại thành context. Số lượng chunks được chọn phụ thuộc vào context window của LLM và độ phức tạp của câu hỏi.

8. Response Generation
Cuối cùng, LLM sử dụng context đã được retrieve cùng với original query để generate ra câu trả lời chính xác và có căn cứ.

Ưu điểm của RAG

RAG mang lại nhiều lợi ích quan trọng:
- Cải thiện độ chính xác: Câu trả lời được base trên thông tin thực tế từ documents
- Giảm hallucination: LLM ít có khả năng tạo ra thông tin sai lệch
- Cập nhật thông tin: Có thể easily update knowledge base mà không cần retrain model
- Traceability: Có thể trace back câu trả lời đến nguồn gốc thông tin
- Cost-effective: Không cần fine-tune LLM với domain-specific data

Thách thức và Giải pháp

Tuy nhiên, RAG cũng đối mặt với một số thách thức:
- Chất lượng retrieval: Cần optimize embedding models và search algorithms
- Chunk size optimization: Balance giữa semantic coherence và retrieval precision
- Latency: Multiple steps có thể tăng response time
- Context management: Quản lý context window limits của LLMs

Để giải quyết các thách thức này, các practitioners thường áp dụng các techniques như re-ranking, query expansion, và hybrid search approaches.
"""

# Tạo Document object
sample_doc = Document(
    page_content=sample_text,
    metadata={"source": "rag_tutorial", "author": "LangChain Tutorial"}
)

print(f"Sample text length: {len(sample_text)} characters")
print(f"Word count: {len(sample_text.split())} words")
print(f"Line count: {len(sample_text.split(chr(10)))} lines")

## 1. CharacterTextSplitter - Splitter cơ bản

In [ ]:
# CharacterTextSplitter cơ bản
char_splitter = CharacterTextSplitter(
    separator="\n\n",  # Chia theo paragraph
    chunk_size=500,   # Kích thước mỗi chunk (characters)
    chunk_overlap=50, # Overlap giữa các chunks
    length_function=len,  # Function để đo độ dài
    is_separator_regex=False  # Separator là string literal, không phải regex
)

# Split document
char_chunks = char_splitter.split_documents([sample_doc])

print(f"CharacterTextSplitter results:")
print(f"Number of chunks: {len(char_chunks)}")
print(f"\nChunk sizes:")
for i, chunk in enumerate(char_chunks):
    print(f"Chunk {i+1}: {len(chunk.page_content)} characters")

print(f"\n=== First 3 chunks ===")
for i, chunk in enumerate(char_chunks[:3]):
    print(f"\n--- Chunk {i+1} ---")
    print(chunk.page_content[:200] + "...")
    print(f"Metadata: {chunk.metadata}")

In [ ]:
# Thử với separator khác
char_splitter_newline = CharacterTextSplitter(
    separator="\n",     # Chia theo line breaks
    chunk_size=300,
    chunk_overlap=30
)

char_chunks_newline = char_splitter_newline.split_documents([sample_doc])

print(f"\nCharacterTextSplitter với separator='\\n':")
print(f"Number of chunks: {len(char_chunks_newline)}")

# So sánh kích thước chunks
sizes_paragraph = [len(chunk.page_content) for chunk in char_chunks]
sizes_newline = [len(chunk.page_content) for chunk in char_chunks_newline]

print(f"\nChunk size comparison:")
print(f"Paragraph separator - Average: {np.mean(sizes_paragraph):.1f}, Std: {np.std(sizes_paragraph):.1f}")
print(f"Newline separator - Average: {np.mean(sizes_newline):.1f}, Std: {np.std(sizes_newline):.1f}")

## 2. RecursiveCharacterTextSplitter - Splitter thông minh

In [ ]:
# RecursiveCharacterTextSplitter - default settings
recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    # Default separators: ["\n\n", "\n", " ", ""]
    separators=None  # Sử dụng default separators
)

recursive_chunks = recursive_splitter.split_documents([sample_doc])

print(f"RecursiveCharacterTextSplitter results:")
print(f"Number of chunks: {len(recursive_chunks)}")
print(f"\nChunk sizes:")
for i, chunk in enumerate(recursive_chunks):
    print(f"Chunk {i+1}: {len(chunk.page_content)} characters")

print(f"\n=== First 3 chunks ===")
for i, chunk in enumerate(recursive_chunks[:3]):
    print(f"\n--- Chunk {i+1} ---")
    print(chunk.page_content)
    print(f"Length: {len(chunk.page_content)} chars")
    print("-" * 50)

In [ ]:
# Custom separators cho Vietnamese text
vietnamese_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=40,
    separators=[
        "\n\n",  # Paragraph breaks
        "\n",    # Line breaks
        ". ",    # Sentence endings
        ", ",    # Clause separators
        " ",     # Word boundaries
        ""       # Character level (last resort)
    ]
)

vietnamese_chunks = vietnamese_splitter.split_documents([sample_doc])

print(f"Vietnamese-optimized splitter:")
print(f"Number of chunks: {len(vietnamese_chunks)}")

# Analyze where splits occur
print(f"\n=== Split analysis ===")
for i, chunk in enumerate(vietnamese_chunks[:4]):
    # Check how chunk starts and ends
    start_char = chunk.page_content[0] if chunk.page_content else ''
    end_chars = chunk.page_content[-5:] if len(chunk.page_content) >= 5 else chunk.page_content
    
    print(f"\nChunk {i+1}:")
    print(f"  Length: {len(chunk.page_content)} chars")
    print(f"  Starts with: '{start_char}'")
    print(f"  Ends with: '{end_chars}'")
    print(f"  Preview: {chunk.page_content[:100]}...")

## 3. Tham số chunk_size và chunk_overlap

In [ ]:
# Thử nghiệm với các chunk_size khác nhau
chunk_sizes = [200, 400, 600, 800]
overlap_ratio = 0.1  # 10% overlap

results = {}

for size in chunk_sizes:
    overlap = int(size * overlap_ratio)
    
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=size,
        chunk_overlap=overlap,
        length_function=len
    )
    
    chunks = splitter.split_documents([sample_doc])
    
    results[size] = {
        'num_chunks': len(chunks),
        'avg_size': np.mean([len(c.page_content) for c in chunks]),
        'size_std': np.std([len(c.page_content) for c in chunks]),
        'overlap_used': overlap
    }

print("Chunk size analysis:")
print(f"{'Size':<6} {'Overlap':<7} {'Chunks':<6} {'Avg Size':<9} {'Std Dev':<8}")
print("-" * 45)

for size, stats in results.items():
    print(f"{size:<6} {stats['overlap_used']:<7} {stats['num_chunks']:<6} "
          f"{stats['avg_size']:<9.1f} {stats['size_std']:<8.1f}")

In [ ]:
# Thử nghiệm với các overlap ratios khác nhau
overlap_ratios = [0, 0.05, 0.1, 0.2, 0.3]
fixed_chunk_size = 500

overlap_results = {}

for ratio in overlap_ratios:
    overlap = int(fixed_chunk_size * ratio)
    
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=fixed_chunk_size,
        chunk_overlap=overlap,
        length_function=len
    )
    
    chunks = splitter.split_documents([sample_doc])
    
    # Calculate actual overlap
    actual_overlaps = []
    for i in range(len(chunks) - 1):
        current_chunk = chunks[i].page_content
        next_chunk = chunks[i + 1].page_content
        
        # Find overlap (simplified approach)
        overlap_chars = 0
        for j in range(1, min(len(current_chunk), len(next_chunk)) + 1):
            if current_chunk[-j:] == next_chunk[:j]:
                overlap_chars = j
        actual_overlaps.append(overlap_chars)
    
    overlap_results[ratio] = {
        'num_chunks': len(chunks),
        'overlap_setting': overlap,
        'avg_actual_overlap': np.mean(actual_overlaps) if actual_overlaps else 0
    }

print(f"\nOverlap analysis (chunk_size={fixed_chunk_size}):")
print(f"{'Ratio':<6} {'Setting':<8} {'Chunks':<6} {'Actual Avg':<12}")
print("-" * 35)

for ratio, stats in overlap_results.items():
    print(f"{ratio:<6} {stats['overlap_setting']:<8} {stats['num_chunks']:<6} "
          f"{stats['avg_actual_overlap']:<12.1f}")

## 4. Visualizing chunk overlap

In [ ]:
# Tạo visualization để hiểu overlap
def visualize_chunks_overlap(chunks, max_chunks=5):
    """Visualize how chunks overlap"""
    print("=== CHUNK OVERLAP VISUALIZATION ===")
    
    for i in range(min(len(chunks) - 1, max_chunks - 1)):
        current = chunks[i].page_content
        next_chunk = chunks[i + 1].page_content
        
        print(f"\nChunk {i+1} -> Chunk {i+2}:")
        print(f"Chunk {i+1} ending: ...{current[-100:]}")
        print(f"Chunk {i+2} starting: {next_chunk[:100]}...")
        
        # Find potential overlap
        overlap_found = False
        for length in range(50, 5, -5):  # Check overlap từ 50 chars xuống 5
            if current[-length:] in next_chunk[:length*2]:
                overlap_text = current[-length:]
                print(f"OVERLAP DETECTED ({length} chars): '{overlap_text[:50]}...'")
                overlap_found = True
                break
        
        if not overlap_found:
            print("No significant overlap detected")
        
        print("-" * 80)

# Test overlap visualization
test_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
)

test_chunks = test_splitter.split_documents([sample_doc])
visualize_chunks_overlap(test_chunks, max_chunks=3)

## 5. So sánh CharacterTextSplitter vs RecursiveCharacterTextSplitter

In [ ]:
# So sánh trực tiếp hai loại splitter
comparison_chunk_size = 400
comparison_overlap = 50

# CharacterTextSplitter
char_splitter_comp = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=comparison_chunk_size,
    chunk_overlap=comparison_overlap
)

# RecursiveCharacterTextSplitter
recursive_splitter_comp = RecursiveCharacterTextSplitter(
    chunk_size=comparison_chunk_size,
    chunk_overlap=comparison_overlap
)

char_chunks_comp = char_splitter_comp.split_documents([sample_doc])
recursive_chunks_comp = recursive_splitter_comp.split_documents([sample_doc])

print("=== COMPARISON: CharacterTextSplitter vs RecursiveCharacterTextSplitter ===")
print(f"Settings: chunk_size={comparison_chunk_size}, chunk_overlap={comparison_overlap}")
print(f"\nCharacterTextSplitter:")
print(f"  - Number of chunks: {len(char_chunks_comp)}")
print(f"  - Average chunk size: {np.mean([len(c.page_content) for c in char_chunks_comp]):.1f}")
print(f"  - Size std deviation: {np.std([len(c.page_content) for c in char_chunks_comp]):.1f}")

print(f"\nRecursiveCharacterTextSplitter:")
print(f"  - Number of chunks: {len(recursive_chunks_comp)}")
print(f"  - Average chunk size: {np.mean([len(c.page_content) for c in recursive_chunks_comp]):.1f}")
print(f"  - Size std deviation: {np.std([len(c.page_content) for c in recursive_chunks_comp]):.1f}")

# Show example chunks
print(f"\n=== CHUNK EXAMPLES ===")
print(f"\nCharacterTextSplitter - Chunk 1:")
print(char_chunks_comp[0].page_content[:300] + "...")

print(f"\nRecursiveCharacterTextSplitter - Chunk 1:")
print(recursive_chunks_comp[0].page_content[:300] + "...")

In [ ]:
# Analyze semantic coherence
def analyze_chunk_endings(chunks, splitter_name):
    """Analyze how chunks end - complete sentences vs cut-off"""
    complete_sentences = 0
    cut_off_sentences = 0
    
    print(f"\n=== {splitter_name} - Ending Analysis ===")
    
    for i, chunk in enumerate(chunks[:5]):  # Analyze first 5 chunks
        content = chunk.page_content.strip()
        if not content:
            continue
            
        last_char = content[-1]
        last_sentence = content.split('.')[-1].strip()
        
        if last_char in '.!?':
            complete_sentences += 1
            ending_type = "Complete sentence"
        else:
            cut_off_sentences += 1
            ending_type = "Cut-off"
        
        print(f"Chunk {i+1}: {ending_type}")
        print(f"  Last 80 chars: ...{content[-80:]}")
        print(f"  Ends with: '{last_char}'")
        print()
    
    return complete_sentences, cut_off_sentences

# Analyze both splitters
char_complete, char_cutoff = analyze_chunk_endings(char_chunks_comp, "CharacterTextSplitter")
recursive_complete, recursive_cutoff = analyze_chunk_endings(recursive_chunks_comp, "RecursiveCharacterTextSplitter")

print(f"\n=== SEMANTIC COHERENCE SUMMARY ===")
print(f"CharacterTextSplitter:")
print(f"  - Complete sentences: {char_complete}")
print(f"  - Cut-off sentences: {char_cutoff}")
print(f"  - Coherence score: {char_complete/(char_complete+char_cutoff)*100:.1f}%")

print(f"\nRecursiveCharacterTextSplitter:")
print(f"  - Complete sentences: {recursive_complete}")
print(f"  - Cut-off sentences: {recursive_cutoff}")
print(f"  - Coherence score: {recursive_complete/(recursive_complete+recursive_cutoff)*100:.1f}%")

## 6. TokenTextSplitter - Split theo tokens

In [ ]:
# TokenTextSplitter sử dụng tiktoken (OpenAI tokenizer)
try:
    token_splitter = TokenTextSplitter(
        encoding_name="cl100k_base",  # GPT-4 encoding
        chunk_size=200,  # 200 tokens per chunk
        chunk_overlap=20
    )
    
    token_chunks = token_splitter.split_documents([sample_doc])
    
    print(f"TokenTextSplitter results:")
    print(f"Number of chunks: {len(token_chunks)}")
    
    # Analyze token counts
    encoding = tiktoken.get_encoding("cl100k_base")
    
    print(f"\nToken analysis:")
    for i, chunk in enumerate(token_chunks[:3]):
        tokens = encoding.encode(chunk.page_content)
        print(f"Chunk {i+1}:")
        print(f"  Characters: {len(chunk.page_content)}")
        print(f"  Tokens: {len(tokens)}")
        print(f"  Chars/Token ratio: {len(chunk.page_content)/len(tokens):.2f}")
        print(f"  Preview: {chunk.page_content[:100]}...")
        print()
        
except ImportError:
    print("tiktoken not installed. Install with: pip install tiktoken")
    
    # Alternative: count "words" as proxy for tokens
    print("Using word-based approximation instead:")
    
    def count_words(text):
        return len(text.split())
    
    word_splitter = RecursiveCharacterTextSplitter(
        chunk_size=150,  # Approximate 200 tokens ≈ 150 words
        chunk_overlap=15,
        length_function=count_words  # Use word count instead of character count
    )
    
    word_chunks = word_splitter.split_documents([sample_doc])
    
    print(f"Word-based splitting:")
    print(f"Number of chunks: {len(word_chunks)}")
    for i, chunk in enumerate(word_chunks[:3]):
        print(f"Chunk {i+1}: {count_words(chunk.page_content)} words, {len(chunk.page_content)} chars")

## 7. Best Practices và Recommendations

In [ ]:
# Function để test và recommend optimal settings
def recommend_chunk_settings(document, target_chunks=None, max_chunk_size=1000):
    """Recommend optimal chunk settings based on document characteristics"""
    doc_length = len(document.page_content)
    word_count = len(document.page_content.split())
    
    print(f"=== DOCUMENT ANALYSIS ===")
    print(f"Document length: {doc_length:,} characters")
    print(f"Word count: {word_count:,} words")
    print(f"Average word length: {doc_length/word_count:.1f} chars/word")
    
    # Analyze document structure
    paragraphs = document.page_content.split('\n\n')
    sentences = document.page_content.split('. ')
    
    avg_paragraph_length = np.mean([len(p) for p in paragraphs if p.strip()])
    avg_sentence_length = np.mean([len(s) for s in sentences if s.strip()])
    
    print(f"\n=== STRUCTURE ANALYSIS ===")
    print(f"Paragraphs: {len(paragraphs)}")
    print(f"Average paragraph length: {avg_paragraph_length:.0f} chars")
    print(f"Sentences: {len(sentences)}")
    print(f"Average sentence length: {avg_sentence_length:.0f} chars")
    
    # Make recommendations
    print(f"\n=== RECOMMENDATIONS ===")
    
    if target_chunks:
        recommended_size = doc_length // target_chunks
        print(f"For {target_chunks} chunks: chunk_size ≈ {recommended_size}")
    else:
        # Base recommendations on document structure
        if avg_paragraph_length > 800:
            recommended_size = int(avg_paragraph_length * 0.8)
            print(f"Large paragraphs detected. Recommended chunk_size: {recommended_size}")
        elif avg_paragraph_length < 200:
            recommended_size = int(avg_paragraph_length * 3)
            print(f"Small paragraphs detected. Recommended chunk_size: {recommended_size}")
        else:
            recommended_size = int(avg_paragraph_length * 1.5)
            print(f"Medium paragraphs. Recommended chunk_size: {recommended_size}")
    
    # Overlap recommendation
    recommended_overlap = max(20, int(recommended_size * 0.1))
    print(f"Recommended overlap: {recommended_overlap} (10% of chunk_size)")
    
    # Splitter recommendation
    if avg_sentence_length > 100:
        print(f"Long sentences detected. Recommend RecursiveCharacterTextSplitter")
        splitter_rec = "RecursiveCharacterTextSplitter"
    else:
        print(f"Well-structured text. CharacterTextSplitter might work well")
        splitter_rec = "CharacterTextSplitter"
    
    return {
        'chunk_size': min(recommended_size, max_chunk_size),
        'chunk_overlap': recommended_overlap,
        'splitter_type': splitter_rec
    }

# Get recommendations
recommendations = recommend_chunk_settings(sample_doc, target_chunks=8)

# Test recommended settings
print(f"\n=== TESTING RECOMMENDATIONS ===")
test_splitter = RecursiveCharacterTextSplitter(
    chunk_size=recommendations['chunk_size'],
    chunk_overlap=recommendations['chunk_overlap']
)

test_chunks = test_splitter.split_documents([sample_doc])
print(f"Result: {len(test_chunks)} chunks created")
print(f"Average chunk size: {np.mean([len(c.page_content) for c in test_chunks]):.0f} chars")

## 8. Use Case Specific Examples

In [ ]:
# Example 1: Code documentation
code_doc_text = """# API Documentation

## Authentication
All API requests must include an API key in the header:
```
Authorization: Bearer YOUR_API_KEY
```

## Endpoints

### GET /users
Retrieve a list of all users.

Parameters:
- limit (optional): Number of users to return (default: 50)
- offset (optional): Number of users to skip (default: 0)

Response:
```json
{
  "users": [
    {
      "id": 1,
      "name": "John Doe",
      "email": "john@example.com"
    }
  ],
  "total": 100
}
```

### POST /users
Create a new user.

Request body:
```json
{
  "name": "Jane Doe",
  "email": "jane@example.com",
  "password": "secure_password"
}
```

Response:
```json
{
  "id": 2,
  "name": "Jane Doe",
  "email": "jane@example.com",
  "created_at": "2024-01-15T10:30:00Z"
}
```
"""

code_doc = Document(page_content=code_doc_text, metadata={"type": "api_docs"})

# Splitter cho code documentation - preserve code blocks
code_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=50,
    separators=[
        "\n## ",    # Section headers
        "\n### ",   # Subsection headers
        "\n\n",     # Paragraphs
        "\n",       # Lines
        " ",        # Words
        "",         # Characters
    ]
)

code_chunks = code_splitter.split_documents([code_doc])

print(f"Code documentation splitting:")
print(f"Chunks created: {len(code_chunks)}")
print(f"\nFirst chunk:")
print(code_chunks[0].page_content)
print(f"\nSecond chunk:")
print(code_chunks[1].page_content if len(code_chunks) > 1 else "No second chunk")

In [ ]:
# Example 2: Legal documents
legal_text = """ĐIỀU KHOẢN SỬ DỤNG DỊCH VỤ

Điều 1. Định nghĩa
1.1. "Dịch vụ" có nghĩa là tất cả các dịch vụ được cung cấp thông qua nền tảng của chúng tôi.
1.2. "Người dùng" là cá nhân hoặc tổ chức sử dụng Dịch vụ.
1.3. "Tài khoản" là tài khoản được tạo bởi Người dùng để truy cập Dịch vụ.

Điều 2. Quyền và nghĩa vụ của Người dùng
2.1. Người dùng có quyền:
a) Sử dụng Dịch vụ theo đúng mục đích được thiết kế;
b) Được bảo vệ thông tin cá nhân theo quy định pháp luật;
c) Khiếu nại khi có vi phạm từ phía nhà cung cấp dịch vụ.

2.2. Người dùng có nghĩa vụ:
a) Cung cấp thông tin chính xác khi đăng ký;
b) Bảo mật thông tin tài khoản;
c) Không sử dụng Dịch vụ cho mục đích bất hợp pháp;
d) Thanh toán đầy đủ các khoản phí theo quy định.

Điều 3. Trách nhiệm của nhà cung cấp dịch vụ
3.1. Cung cấp Dịch vụ ổn định và chất lượng.
3.2. Bảo vệ thông tin cá nhân của Người dùng.
3.3. Hỗ trợ Người dùng khi gặp sự cố kỹ thuật.
"""

legal_doc = Document(page_content=legal_text, metadata={"type": "legal_terms"})

# Splitter cho văn bản pháp lý - preserve article structure
legal_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=30,
    separators=[
        "\n\nĐiều ",  # Articles
        "\n\n",       # Paragraphs
        "\n",         # Lines
        "; ",         # Clauses
        ", ",         # Sub-clauses
        " ",          # Words
        "",           # Characters
    ]
)

legal_chunks = legal_splitter.split_documents([legal_doc])

print(f"\nLegal document splitting:")
print(f"Chunks created: {len(legal_chunks)}")

for i, chunk in enumerate(legal_chunks[:3]):
    print(f"\n--- Legal Chunk {i+1} ---")
    print(chunk.page_content)
    print(f"Length: {len(chunk.page_content)} chars")

## 9. Performance và Memory Considerations

In [ ]:
import time
import sys

# Tạo large document để test performance
large_text = sample_text * 10  # 10x larger
large_doc = Document(page_content=large_text, metadata={"type": "large_doc"})

print(f"Large document stats:")
print(f"- Characters: {len(large_text):,}")
print(f"- Words: {len(large_text.split()):,}")
print(f"- Memory size: {sys.getsizeof(large_text):,} bytes")

# Test different splitters performance
splitters_to_test = {
    "CharacterTextSplitter": CharacterTextSplitter(
        separator="\n\n", chunk_size=500, chunk_overlap=50
    ),
    "RecursiveCharacterTextSplitter": RecursiveCharacterTextSplitter(
        chunk_size=500, chunk_overlap=50
    )
}

performance_results = {}

for name, splitter in splitters_to_test.items():
    start_time = time.time()
    chunks = splitter.split_documents([large_doc])
    end_time = time.time()
    
    performance_results[name] = {
        'time': end_time - start_time,
        'chunks': len(chunks),
        'avg_chunk_size': np.mean([len(c.page_content) for c in chunks])
    }

print(f"\n=== PERFORMANCE COMPARISON ===")
for name, results in performance_results.items():
    print(f"{name}:")
    print(f"  - Time: {results['time']:.4f} seconds")
    print(f"  - Chunks: {results['chunks']}")
    print(f"  - Avg chunk size: {results['avg_chunk_size']:.0f} chars")
    print(f"  - Chunks per second: {results['chunks']/results['time']:.1f}")
    print()

## 10. Common Pitfalls và Troubleshooting

In [ ]:
# Common pitfall 1: Chunk size quá nhỏ
print("=== PITFALL 1: Chunk size quá nhỏ ===")
tiny_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,  # Quá nhỏ!
    chunk_overlap=10
)

tiny_chunks = tiny_splitter.split_documents([sample_doc])
print(f"Chunks with tiny size: {len(tiny_chunks)}")
print(f"First tiny chunk: '{tiny_chunks[0].page_content}'")
print(f"Problem: Chunks không có đủ context để meaningful retrieval")

# Common pitfall 2: Overlap quá lớn
print(f"\n=== PITFALL 2: Overlap quá lớn ===")
huge_overlap_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=150  # 75% overlap!
)

huge_overlap_chunks = huge_overlap_splitter.split_documents([sample_doc])
print(f"Chunks with huge overlap: {len(huge_overlap_chunks)}")
print(f"Problem: Quá nhiều duplicate content, tăng cost và noise")

# Common pitfall 3: Ignoring document structure
print(f"\n=== PITFALL 3: Ignoring document structure ===")
bad_separator_splitter = CharacterTextSplitter(
    separator=" ",  # Split by spaces - very bad!
    chunk_size=200,
    chunk_overlap=20
)

bad_chunks = bad_separator_splitter.split_documents([sample_doc])
print(f"Chunks with bad separator: {len(bad_chunks)}")
print(f"First bad chunk: '{bad_chunks[0].page_content}'")
print(f"Problem: Chunks không respect ngữ nghĩa và structure")

# Solution examples
print(f"\n=== SOLUTIONS ===")
print(f"1. Optimal chunk size: 200-800 characters cho general text")
print(f"2. Overlap ratio: 10-20% of chunk size")
print(f"3. Use RecursiveCharacterTextSplitter với appropriate separators")
print(f"4. Test với actual data để tune parameters")

## Tổng kết

### **Tại sao cần Text Splitting?**
1. **Context window limits** của LLMs
2. **Retrieval efficiency** - chunks nhỏ hơn = precision cao hơn
3. **Cost optimization** - chỉ process relevant parts
4. **Semantic coherence** - mỗi chunk một ý tưởng

### **CharacterTextSplitter vs RecursiveCharacterTextSplitter**

| Aspect | CharacterTextSplitter | RecursiveCharacterTextSplitter |
|--------|----------------------|--------------------------------|
| **Flexibility** | Single separator | Multiple separators, hierarchical |
| **Semantic Preservation** | Có thể cắt ngang câu | Cố gắng preserve semantic boundaries |
| **Performance** | Nhanh hơn | Chậm hơn một chút |
| **Use Cases** | Well-structured documents | General purpose, mixed content |
| **Recommendation** | Khi biết rõ document structure | Default choice cho most cases |

### **Key Parameters**
- **`chunk_size`**: 200-800 chars cho general text, 100-300 tokens cho LLM processing
- **`chunk_overlap`**: 10-20% của chunk_size để maintain context
- **`separators`**: Hierarchical từ paragraph → sentence → word → character

### **Best Practices**
1. **Test với actual data** để tìm optimal settings
2. **Analyze document structure** trước khi chọn splitter
3. **Monitor chunk quality** - check endings, semantic coherence
4. **Consider domain-specific requirements** (code, legal, scientific texts)
5. **Balance chunk size với retrieval precision**

### **Next Steps**
- **Embeddings**: Convert chunks thành vectors
- **Vector Stores**: Store và search embeddings
- **Retrieval**: Find relevant chunks for queries
- **Generation**: Use retrieved context với LLMs

Text Splitting là foundation quan trọng cho successful RAG implementations!